# Final Year Project

In [1]:
# importing libraries
import requests
import os
import shutil
import pandas as pd

In [2]:
# Data scrapped from the range of years 
years = list(range(2003,2022))

In [3]:
from bs4 import BeautifulSoup

# Main Datasets

Playoff_data is taken from https://github.com/trustinyoon/2020-NBA-Chip-Predictor this dataset gives us the stats for the off season or you can call it the playoffs.

predict.csv is taken from http://basketball-reference.com/ which is web scrapped by me. This dataset gives us stats from the regular season where teams compete to qualify for the playoffs.

# Load Dataset

In [104]:
chip = pd.read_csv("predict.csv")
chip

,Unnamed: 0,Tm,W,L,W/L%,PW,PL,ORtg,DRtg,NRtg,...,eFG%,TOV%,ORB%,FT/FGA,Unnamed: 20,eFG%.1,TOV%.1,DRB%,FT/FGA.1,Year
0,0,San Antonio Spurs,16.0,8.0,0.667,17,7,103.7,97.7,6.0,...,0.476,14.4,28.6,0.284,NaN,0.439,14.0,73.4,0.216,2003
1,1,New Jersey Nets,14.0,6.0,0.700,12,8,102.6,98.9,3.7,...,0.453,14.1,30.9,0.256,NaN,0.464,14.8,76.7,0.252,2003
2,2,Sacramento Kings,7.0,5.0,0.583,7,5,112.0,109.6,2.4,...,0.499,12.0,29.3,0.265,NaN,0.495,12.7,70.7,0.216,2003
3,3,Portland Trail Blazers,3.0,4.0,0.429,4,3,115.2,114.1,1.1,...,0.501,13.0,36.3,0.280,NaN,0.507,11.1,72.7,0.228,2003
4,4,Detroit Pistons,8.0,9.0,0.471,9,8,102.7,101.7,1.0,...,0.455,13.3,25.5,0.308,NaN,0.459,14.8,70.6,0.253,2003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,11,Utah Jazz,2.0,4.0,0.333,2,4,108.8,115.0,-6.2,...,0.495,11.5,22.8,0.264,NaN,0.540,8.7,85.5,0.217,2022
316,12,Denver Nuggets,1.0,4.0,0.200,1,4,113.6,121.8,-8.2,...,0.543,14.9,28.0,0.242,NaN,0.608,12.2,81.4,0.230,2022
317,13,Toronto Raptors,2.0,4.0,0.333,2,4,111.2,120.2,-9.0,...,0.503,9.5,23.0,0.194,NaN,0.576,14.0,76.7,0.275,2022
318,14,Atlanta Hawks,1.0,4.0,0.200,1,4,102.3,114.9,-12.6,...,0.513,15.7,21.3,0.220,NaN,0.542,12.2,79.0,0.194,2022


# Clean DataSet

In [112]:
chip.columns

Index(['Tm', 'W', 'L', 'W/L%', 'PW', 'PL', 'ORtg', 'DRtg', 'NRtg', 'Pace',
       'FTr', '3PAr', 'TS%', 'eFG%', 'TOV%', 'ORB%', 'FT/FGA', 'eFG%.1',
       'TOV%.1', 'DRB%', 'FT/FGA.1', 'Year'],
      dtype='object')

In [111]:
chip = chip[chip["Tm"] != 'League Average']
del chip["Unnamed: 20"]
chip

,Tm,W,L,W/L%,PW,PL,ORtg,DRtg,NRtg,Pace,...,TS%,eFG%,TOV%,ORB%,FT/FGA,eFG%.1,TOV%.1,DRB%,FT/FGA.1,Year
0,San Antonio Spurs,16.0,8.0,0.667,17,7,103.7,97.7,6.0,91.0,...,0.525,0.476,14.4,28.6,0.284,0.439,14.0,73.4,0.216,2003
1,New Jersey Nets,14.0,6.0,0.700,12,8,102.6,98.9,3.7,89.9,...,0.506,0.453,14.1,30.9,0.256,0.464,14.8,76.7,0.252,2003
2,Sacramento Kings,7.0,5.0,0.583,7,5,112.0,109.6,2.4,94.7,...,0.549,0.499,12.0,29.3,0.265,0.495,12.7,70.7,0.216,2003
3,Portland Trail Blazers,3.0,4.0,0.429,4,3,115.2,114.1,1.1,87.9,...,0.551,0.501,13.0,36.3,0.280,0.507,11.1,72.7,0.228,2003
4,Detroit Pistons,8.0,9.0,0.471,9,8,102.7,101.7,1.0,86.5,...,0.515,0.455,13.3,25.5,0.308,0.459,14.8,70.6,0.253,2003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,Utah Jazz,2.0,4.0,0.333,2,4,108.8,115.0,-6.2,91.0,...,0.546,0.495,11.5,22.8,0.264,0.540,8.7,85.5,0.217,2022
316,Denver Nuggets,1.0,4.0,0.200,1,4,113.6,121.8,-8.2,96.9,...,0.586,0.543,14.9,28.0,0.242,0.608,12.2,81.4,0.230,2022
317,Toronto Raptors,2.0,4.0,0.333,2,4,111.2,120.2,-9.0,91.2,...,0.542,0.503,9.5,23.0,0.194,0.576,14.0,76.7,0.275,2022
318,Atlanta Hawks,1.0,4.0,0.200,1,4,102.3,114.9,-12.6,95.2,...,0.554,0.513,15.7,21.3,0.220,0.542,12.2,79.0,0.194,2022


In [37]:
playoff_data = pd.read_csv("playoffs.csv")
playoff_data

,Unnamed: 0,Rk,Team,Year,Playoff Wins,MP,FG,FGA,FG%,3P,...,SRS,ORtg,DRtg,Pace,eFG%,TOV%,ORB%,eFG%.1,DRB%,FT/FGA.1
0,0,1.0,San Antonio Spurs,2003,16.0,14,16,27,4,11,...,3,7,3,20,4,24,13,2,17,3
1,1,2.0,New Jersey Nets,2003,14.0,26,17,18,14,23,...,4,18,1,11,21,16,14,4,6,14
2,2,3.0,Dallas Mavericks,2003,10.0,24,3,2,7,2,...,1,1,9,7,3,1,27,15,19,10
3,3,4.0,Detroit Pistons,2003,8.0,5,26,28,24,6,...,5,15,4,29,14,9,24,5,1,15
4,4,5.0,Sacramento Kings,2003,7.0,14,1,1,3,7,...,2,6,2,1,2,6,26,1,21,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,289,NaN,Portland Trail Blazers,2020,NaN,5,8,6,15,11,...,18,10,27,12,16,4,19,14,27,17
290,290,NaN,New Orleans Pelicans,2020,NaN,20,4,1,13,3,...,14,15,18,2,7,27,5,17,11,18
291,291,NaN,Sacramento Kings,2020,NaN,20,19,23,17,10,...,21,19,19,25,13,16,21,19,7,26
292,292,NaN,San Antonio Spurs,2020,NaN,29,6,13,8,26,...,20,11,25,15,17,1,29,23,4,13


# Cleaning Datasets

In [38]:
#inspecting the different team stat categories
playoff_data.columns

Index(['Unnamed: 0', 'Rk', 'Team', 'Year', 'Playoff Wins', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'O_MP',
       'W', 'L', 'PW', 'PL', 'MOV', 'SOS', 'SRS', 'ORtg', 'DRtg', 'Pace',
       'eFG%', 'TOV%', 'ORB%', 'eFG%.1', 'DRB%', 'FT/FGA.1'],
      dtype='object')

In [48]:
del playoff_data["Unnamed: 0"]

In [49]:
del playoff_data["Rk"]

In [83]:
playoff_data.columns

Index(['Team', 'Year', 'Playoff Wins', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA',
       '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'O_MP', 'W', 'L', 'PW', 'PL',
       'MOV', 'SOS', 'SRS', 'ORtg', 'DRtg', 'Pace', 'eFG%', 'TOV%', 'ORB%',
       'eFG%.1', 'DRB%', 'FT/FGA.1'],
      dtype='object')

In [84]:
playoff_data.fillna(method="ffill")

,Team,Year,Playoff Wins,MP,FG,FGA,FG%,3P,3PA,3P%,...,SRS,ORtg,DRtg,Pace,eFG%,TOV%,ORB%,eFG%.1,DRB%,FT/FGA.1
0,San Antonio Spurs,2003,16.0,14,16,27,4,11,11,11,...,3,7,3,20,4,24,13,2,17,3
1,New Jersey Nets,2003,14.0,26,17,18,14,23,22,22,...,4,18,1,11,21,16,14,4,6,14
2,Dallas Mavericks,2003,10.0,24,3,2,7,2,2,3,...,1,1,9,7,3,1,27,15,19,10
3,Detroit Pistons,2003,8.0,5,26,28,24,6,6,8,...,5,15,4,29,14,9,24,5,1,15
4,Sacramento Kings,2003,7.0,14,1,1,3,7,9,4,...,2,6,2,1,2,6,26,1,21,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,Portland Trail Blazers,2020,0.0,5,8,6,15,11,15,4,...,18,10,27,12,16,4,19,14,27,17
290,New Orleans Pelicans,2020,0.0,20,4,1,13,3,6,5,...,14,15,18,2,7,27,5,17,11,18
291,Sacramento Kings,2020,0.0,20,19,23,17,10,11,11,...,21,19,19,25,13,16,21,19,7,26
292,San Antonio Spurs,2020,0.0,29,6,13,8,26,28,7,...,20,11,25,15,17,1,29,23,4,13


# Split dataset

In [97]:
predictors = ['Year', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA',
       '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'O_MP',
       'MOV', 'SOS', 'SRS', 'ORtg', 'DRtg', 'Pace', 'eFG%', 'TOV%', 'ORB%',
       'eFG%.1', 'DRB%', 'FT/FGA.1']

In [146]:
predictors_chip = ['ORtg', 'DRtg', 'NRtg', 'Pace',
       'FTr', '3PAr', 'TS%', 'eFG%', 'TOV%', 'ORB%', 'FT/FGA', 'eFG%.1',
       'TOV%.1', 'DRB%', 'FT/FGA.1', 'Year']

In [244]:
# splaitting the dataset
train = chip[~(chip["Year"] == 2021)]
test = chip[chip["Year"] == 2021]


# Model fitting [base]

In [245]:
from sklearn.linear_model import Ridge

reg = Ridge(alpha=.1)

In [246]:
reg.fit(train[predictors_chip],train["W"])

Ridge(alpha=0.1)

In [238]:
preds = reg.predict(test[predictors_chip])
preds = pd.DataFrame(preds, columns=["preds"], index=test.index)

In [247]:
pred_table = pd.concat([test[["Tm", "W"]], preds], axis=1)
pred_table.sort_values("preds", ascending=False).head(10)

,Tm,W,preds
289,Brooklyn Nets,7.0,10.062163
288,Philadelphia 76ers,7.0,9.940037
292,Los Angeles Clippers,10.0,9.529132
290,Milwaukee Bucks,16.0,9.497166
291,Phoenix Suns,14.0,8.963387
293,Utah Jazz,6.0,6.679859
294,Portland Trail Blazers,2.0,6.052346
295,Atlanta Hawks,10.0,5.728670
296,Dallas Mavericks,3.0,3.806204
297,Denver Nuggets,4.0,3.623296


In [248]:
pred_table["W"].value_counts()

1.0     4
7.0     2
10.0    2
2.0     2
16.0    1
14.0    1
6.0     1
3.0     1
4.0     1
0.0     1
Name: W, dtype: int64

In [249]:
actual = pred_table.sort_values("W", ascending=False)
predicted = pred_table.sort_values("preds", ascending=False)
actual["Rk"] = list(range(1,actual.shape[0]+1))
predicted["Predicted_Rk"] = list(range(1,predicted.shape[0]+1))

In [250]:
actual.merge(predicted, on="Tm").head(5)

,Tm,W_x,preds_x,Rk,W_y,preds_y,Predicted_Rk
0,Milwaukee Bucks,16.0,9.497166,1,16.0,9.497166,4
1,Phoenix Suns,14.0,8.963387,2,14.0,8.963387,5
2,Los Angeles Clippers,10.0,9.529132,3,10.0,9.529132,3
3,Atlanta Hawks,10.0,5.728670,4,10.0,5.728670,8
4,Philadelphia 76ers,7.0,9.940037,5,7.0,9.940037,2


In [255]:
aps = []
all_predictions = []
for year in years[5:]:
    train = chip[chip["Year"] < year]
    test = chip[chip["Year"] == year]
    reg.fit(train[predictors_chip],train["W"])
    predictions = reg.predict(test[predictors_chip])
    predictions = pd.DataFrame(predictions, columns=["predictions"], index=test.index)
    pred_table = pd.concat([test[["Tm", "W", "Year"]], predictions], axis=1)
    all_predictions.append(pred_table)

In [256]:
def add_ranks(predictions):
    predictions = predictions.sort_values("predictions", ascending=False)
    predictions["Predicted_Rk"] = list(range(1,predictions.shape[0]+1))
    predictions = predictions.sort_values("W", ascending=False)
    predictions["Rk"] = list(range(1,predictions.shape[0]+1))
    predictions["Diff"] = (predictions["Rk"] - predictions["Predicted_Rk"])
    return predictions

In [260]:
add_ranks(all_predictions[12])

,Tm,W,Year,predictions,Predicted_Rk,Rk,Diff
272,Los Angeles Lakers,16.0,2020,10.829034,1,1,0
277,Miami Heat,14.0,2020,8.387777,6,2,-4
273,Boston Celtics,10.0,2020,9.190093,4,3,-1
280,Denver Nuggets,9.0,2020,6.154747,9,4,-5
274,Los Angeles Clippers,7.0,2020,9.202400,3,5,2
275,Toronto Raptors,7.0,2020,8.936947,5,6,1
279,Houston Rockets,5.0,2020,7.946897,7,7,0
278,Milwaukee Bucks,5.0,2020,7.547609,8,8,0
276,Utah Jazz,3.0,2020,9.976123,2,9,7
281,Oklahoma City Thunder,3.0,2020,4.157544,10,10,0


In [264]:
def backtest(stats, model, years, predictors):
    aps = []
    all_predictions = []
    for year in years:
        train = chip[chip["Year"] < year]
        test = chip[chip["Year"] == year]
        reg.fit(train[predictors_chip],train["W"])
        predictions = reg.predict(test[predictors_chip])
        predictions = pd.DataFrame(predictions, columns=["predictions"], index=test.index)
        pred_table = pd.concat([test[["Tm", "W", "Year"]], predictions], axis=1)
        pred_table = add_ranks(pred_table)
        all_predictions.append(pred_table)
    return pd.concat(all_predictions)

In [265]:
all_predictions = backtest(chip, reg, years[5:], predictors_chip)

In [266]:
all_predictions[all_predictions["Rk"] < 5].sort_values("Diff").head(50)

,Tm,W,Year,predictions,Predicted_Rk,Rk,Diff
247,Cleveland Cavaliers,12.0,2018,6.125877,8,2,-6
232,Boston Celtics,9.0,2017,5.306967,9,3,-6
200,Houston Rockets,9.0,2015,3.713807,9,3,-6
280,Denver Nuggets,9.0,2020,6.154747,9,4,-5
217,Toronto Raptors,10.0,2016,4.044061,9,4,-5
183,Oklahoma City Thunder,10.0,2014,5.725701,8,4,-4
277,Miami Heat,14.0,2020,8.387777,6,2,-4
263,Portland Trail Blazers,8.0,2019,4.969873,8,4,-4
198,Atlanta Hawks,8.0,2015,5.458555,8,4,-4
295,Atlanta Hawks,10.0,2021,5.660919,8,4,-4


In [161]:
pd.concat([pd.Series(reg.coef_), pd.Series(predictors_chip)], axis=1).sort_values(0, ascending=False)


,0,1
7,3.555876,eFG%
6,3.328422,TS%
5,2.612263,3PAr
4,0.951786,FTr
2,0.324846,NRtg
10,0.189156,FT/FGA
12,0.174934,TOV%.1
0,0.171113,ORtg
8,0.090872,TOV%
13,0.040326,DRB%
